## Creates the Project Points CSVs needed to run ReV

This notebook: 
1. Merges the Zarrs containing individual restrictions into a single Zarr.
2. Uses the resulting merged Zarr to create ReV-ready Project Points CSVs which define the valid grid cells for a given installation. Please see ReV documentation for more information on Project Points.

### Incorporating land use/land cover restrictions

The following table summarizes the relevant land use exclusions for PVWattsv8 or WindPower, respectively, under a given installation type. In other words, if a grid cell meets any of these criteria, it was excluded from the indicated simulations. Note that [PV and wind resource Zarrs in the S3 bucket](https://wfclimres.s3.amazonaws.com/index.html#era/resource_data/) do not have any exclusions, so there exists the potential for interested parties to perform simulations at excluded grid cells should they so desire.

| Restriction | Name in exclusion datasets | Utility PV | Distributed PV | Onshore wind | Offshore wind |
| :--- | :--- | :---: | :---: | :---: | :---: |
| Federal and state protected lands | fed_state_protected | X | X | X | X |
| Slope > 20% | wind_slope_over_20p | X | | X | X |
| High intensity urban | pv_land_cover_urban_high | X | | X | X |
| Medium intensity urban | wind_land_cover_urban_medium_high | | | X | X |
| Open water | landmask | X | X | X | |

* _High intensity urban_ refers to row houses, apartment buildings, and commercial buildings.
* _Medium intensity urban_ refers to single family homes.

#### Original geospatial files

Geospatial files used to mask out grid points for exclusions were sourced from the following:

* Fed/State protected lands [Protected Areas Database of the United States (PAD-US) 3.0 (ver. 2.0, March 2023)](https://www.usgs.gov/programs/gap-analysis-project/science/pad-us-data-download)
* Slope >20% [LandFire, Version 2.2.0, Slope Percent Rise](https://www.landfire.gov/version_download.php#)
* [High intensity urban NLCD](https://www.mrlc.gov/data?f%5B0%5D=category%3ALand%20Cover&f%5B1%5D=region%3Aconus&f%5B2%5D=year%3A2021)
* Open water was masked with the WRF domain’s land-sea mask

#### Derived geospatial datasets

* The geospatial files (.gpkg format) used to create the land use/cover exclusion mask are [archived in the S3 bucket](https://wfclimres.s3.amazonaws.com/index.html#era/resource_data/land_use_restrictions/)
* The gridded datasets with the binary land use/cover restriction masks are also available in the relevant [resource data folders](https://wfclimres.s3.amazonaws.com/index.html#era/resource_data/).

##### Please note: This notebook is dependent on gridded land use/land cover exclusion masks for renewable energy siting.

Here is the workflow we used:

* **Python Preprocessing:** `src/preprocess/all_02_lulc_exclusions.py`
* **SLURM Batch Script:** `scripts/all-02_lulc_exclusions.sbatch`
* **Exclusion List:** `scripts/list-exclusions.dat`

Running the above will pull the .gpkg files from the S3 bucket which correspond to the listed exclusions and convert them to gridded Zarrs which match the coordinates of the WRF datasets. 

In [1]:
import xarray as xr

Create a merged gridded Zarr from the individual land use/land cover Zarrs

In [ ]:
domain = "d02"
zarr_paths = [
    f"s3://wfclimres/era/resource_data/{domain}/static_files/coord_ds_{domain}_fed_state_protected.zarr/",
    f"s3://wfclimres/era/resource_data/{domain}/static_files/coord_ds_{domain}_pv_land_cover_urban_high.zarr/",
    f"s3://wfclimres/era/resource_data/{domain}/static_files/coord_ds_{domain}_wind_land_cover_urban_medium_high.zarr/",
    f"s3://wfclimres/era/resource_data/{domain}/static_files/coord_ds_{domain}_wind_slope_over_20p.zarr/",
    f"s3://wfclimres/era/resource_data/{domain}/static_files/coord_ds_{domain}.zarr/",
]

out_zarr_path = f"s3://wfclimres/era/resource_data/{domain}/static_files/coord_ds_{domain}_all_restrictions.zarr"

In [ ]:
ds_list = []
# Read all Zarrs then merge into one
for p in zarr_paths:
    ds = xr.open_zarr(p)
    ds_list.append(ds)
template_ds = xr.merge(ds_list)
template_ds = template_ds.drop("member_id")
template_ds = template_ds.squeeze()
# Save merged zarr
template_ds.to_zarr(out_zarr_path)

Create a CSV which contains all grid cells - this is necessary for the last postprocessing step

In [ ]:
cols_to_keep = [
    "lat",
    "lon",
    "landmask",
    "pv_land_cover_urban_high",
    "wind_land_cover_urban_medium_high",
    "fed_state_protected",
    "wind_slope_over_20p",
]
df_pp = template_ds.to_dataframe().reset_index()[cols_to_keep]
df_pp["config"] = "def"
display(df_pp)

In [ ]:
# Save csv with all grid points
df_pp.to_csv(f"../rev_configs/{domain}_all_points.csv")

Make project points CSV for the PV simulations

First, make a base one which contains restrictions for both the distributed and utility-scale installations

In [ ]:
# Filter dataframe for base PV site restrictions
pv_pp = (
    df_pp.loc[(df_pp["fed_state_protected"] == 0) & (df_pp["landmask"] == 1)]
    .drop(columns=["lat", "lon"])
    .reset_index()
)
pv_pp = pv_pp.rename(columns={"index": "gid"})
display(pv_pp)

Make the distributed PV Project Points:

In [ ]:
# Distributed PV has only the base restrictions
distributed_pv_pp = pv_pp[["gid", "config"]]
display(distributed_pv_pp)
distributed_pv_pp.to_csv(
    f"../rev_configs/pv_{domain}_distributed_projectpoints.csv", index=False
)

Make the utility-scale PV Project Points:

In [ ]:
# Utility-scale PV has additional restrictions
util_pv_pp = pv_pp.loc[
    (pv_pp["wind_slope_over_20p"] == 0) & (pv_pp["pv_land_cover_urban_high"] == 0)
]
display(util_pv_pp)
util_pv_pp = util_pv_pp[["gid", "config"]]
util_pv_pp.to_csv(f"../rev_configs/pv_{domain}_utility_projectpoints.csv", index=False)

Make Project Points CSVs for the wind power simulations

First, make a base one which contains restrictions for both the onshore and offshore installations

In [ ]:
# Filter dataframe for base wind power site restrictions
wind_pp = df_pp.loc[
    (df_pp["fed_state_protected"] == 0)
    & (df_pp["wind_slope_over_20p"] == 0)
    & (df_pp["wind_land_cover_urban_medium_high"] == 0)
].drop(columns=["lat", "lon"])
display(wind_pp)

Make the onshore Project Points:

In [ ]:
# Filter for onshore wind
onshore_wind_pp = wind_pp.loc[(wind_pp["landmask"] == 1)].reset_index()
onshore_wind_pp = onshore_wind_pp.rename(columns={"index": "gid"})
display(onshore_wind_pp)
onshore_wind_pp = onshore_wind_pp[["gid", "config"]]
onshore_wind_pp.to_csv(
    f"../rev_configs/wind_{domain}_onshore_projectpoints.csv", index=False
)

Make the offshore Project Points:

In [ ]:
# Filter for offshore wind
offshore_wind_pp = wind_pp.loc[(wind_pp["landmask"] == 0)].reset_index()
offshore_wind_pp = offshore_wind_pp.rename(columns={"index": "gid"})
display(offshore_wind_pp)
offshore_wind_pp = offshore_wind_pp[["gid", "config"]]
offshore_wind_pp.to_csv(
    f"../rev_configs/wind_{domain}_offshore_projectpoints.csv", index=False
)